In [1]:
import phoenix as px
from phoenix.trace.langchain import OpenInferenceTracer, LangChainInstrumentor
import pandas as pd
import numpy as np
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from data_handling import *
from embedding import *
from prompt_template import *
from QA  import * 

In [2]:
## SETUP
# Parameter setup
chain_type = "stuff"
chat_model_name='orca-mini'

# Read data from files 
questions = load_json_data('questions.json')
model_answers = load_json_data('model_answers.json')

In [3]:
## FINE TUNING SETUP
# Loading the Embedding Model
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

# Loading and splitting the PDF documents
docs = load_pdf_data(file_path="./../../data/hp1.pdf")
documents = split_docs(documents=docs)

# Creating vectorstore
vectorstore = create_embeddings(documents, embed)

# Converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

# Creating the prompt from the template
prompt=PromptTemplate.from_template(template)

In [4]:
## LAUNCH
# Launch phoenix
session = px.launch_app()

# If no exporter is specified, the tracer will export to the locally running Phoenix server
tracer = OpenInferenceTracer()
LangChainInstrumentor(tracer).instrument()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
# Loading model from Ollama
llm = Ollama(model=chat_model_name, temperature=0)

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [6]:
# Calling the chain
for item in model_answers[chat_model_name]:
    model_answer = item['answer']
    correct_answer = [x['answer'] for x in questions if x['id'] == item['id']]
    print('\n')
    print('Question:\n',item['question'])
    print('Answer (model):\n', model_answer)
    print('Answer (correct):\n', correct_answer[0])
    response = get_response(
       '''first_text: {model_answer}second_text: {correct_answer[0]}''', 
       chain,
       tracer
      )

    print('Evaulation:\n', response)



Question:
 What happened to Harry Potter at the sorting ceremony?
Answer (model):
  As an AI assistant, I do not have enough information about the events that took place during the
sorting ceremony. However, Harry Potter was sorted into Gryffindor House along with Ron Weasley and
Hermione Granger.
Answer (correct):
 After talking with the sorting hat he got sorted into Gryffindor house
Evaulation:
  False.


Question:
 List the names of the Weasley children
Answer (model):
  The names of the Weasley children are Fred, George, Ron, and Ginny.
Answer (correct):
 Ron, Ginny, Fred, George, Charlie and Bill
Evaulation:
  False.


Question:
 What broom does Harry Potter own?
Answer (model):
  Harry Potter owns a Nimbus Two Thousand, which is a broomstick that can fly him through the air.
Answer (correct):
 Nimbus 2000
Evaulation:
  False.


Question:
 List the ingreedients needed for first year students at Hogwarts
Answer (model):
  Here are the ingredients needed for first-year students a